In [1]:
import torch
import torch_geometric
import os 
from utilities_new import GraphDataset
from model.gcn_lstm import GNNPolicy
import time

In [2]:
#set folder

model_save_path = './model_save/1bin_50_pretrain_routine_lstm/'
log_save_path = "./model_save/1bin_50_log_routine_lstm/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'ab')

In [3]:
LEARNING_RATE = 1e-5
NB_EPOCHS =9999
BATCH_SIZE = 2
NUM_WORKERS = 0

In [4]:
train_data = GraphDataset('dataset_1888rte/1bin/train/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=False)
valid_data = GraphDataset('dataset_1888rte/1bin/valid/')
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict)
    print("载入模型成功")


载入模型成功


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            #predict the binary distribution, BD
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
            )
            
            # compute loss
            
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            loss = loss_fn(presol, weighted_sol)

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [8]:
optimizer = torch.optim.Adam(PredictModel.parameters(), lr=LEARNING_RATE)
best_val_loss = 0.15437980539895393

In [9]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.3f}")
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.3f}")
    if valid_loss<best_val_loss:
        best_val_loss = valid_loss
        print(best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}\n'
    log_file.write(st.encode())
    log_file.flush()
print('done')

Epoch 0 Train loss: 0.161
Epoch 0 Valid loss: 0.156
0.15594480355849138
Epoch 1 Train loss: 0.161
Epoch 1 Valid loss: 0.158
Epoch 2 Train loss: 0.161
Epoch 2 Valid loss: 0.158
Epoch 3 Train loss: 0.161
Epoch 3 Valid loss: 0.168
Epoch 4 Train loss: 0.160
Epoch 4 Valid loss: 0.161
Epoch 5 Train loss: 0.160
Epoch 5 Valid loss: 0.155
0.1553801501924927
Epoch 6 Train loss: 0.160
Epoch 6 Valid loss: 0.155
0.15521252839951902
Epoch 7 Train loss: 0.160
Epoch 7 Valid loss: 0.156
Epoch 8 Train loss: 0.160
Epoch 8 Valid loss: 0.155
0.15517162012087332
Epoch 9 Train loss: 0.159
Epoch 9 Valid loss: 0.158
Epoch 10 Train loss: 0.160
Epoch 10 Valid loss: 0.155
Epoch 11 Train loss: 0.159
Epoch 11 Valid loss: 0.154
0.15440459472907556
Epoch 12 Train loss: 0.159
Epoch 12 Valid loss: 0.157
Epoch 13 Train loss: 0.159
Epoch 13 Valid loss: 0.156
Epoch 14 Train loss: 0.166
Epoch 14 Valid loss: 0.161
Epoch 15 Train loss: 0.161
Epoch 15 Valid loss: 0.159
Epoch 16 Train loss: 0.160
Epoch 16 Valid loss: 0.157
Epo